Code to create biochem ICs for SOG model for full Salish Sea domain
- not final; should find climatology of deep values and try to find more measurements for lateral resolution
- all based on S3 from October 2004

In [1]:
import netCDF4 as nc

import matplotlib.pyplot as plt
import numpy as np
import os
import re

%matplotlib inline

#resultsDir='/data/eolson/MEOPAR/SS2DSOGruns/'
N2chl=1.600

In [2]:
TS=nc.Dataset('/results/SalishSea/nowcast/11jun15/SalishSea_02376000_restart.nc')
zvar='nav_lev'
xvar='nav_lon'
yvar='nav_lat'
tvar='time_counter'
tlen=TS.variables['sn'].shape[0]
zlen=TS.variables['sn'].shape[1]
ylen=TS.variables['sn'].shape[2]
xlen=TS.variables['sn'].shape[3]
print(TS.dimensions)
print(TS.variables)

OrderedDict([('t', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 't', size = 1
), ('z', <class 'netCDF4._netCDF4.Dimension'>: name = 'z', size = 40
), ('y', <class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 898
), ('x', <class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 398
)])
OrderedDict([('adatrj', <class 'netCDF4._netCDF4.Variable'>
float64 adatrj()
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of 9.969209968386869e+36 used
), ('avm', <class 'netCDF4._netCDF4.Variable'>
float64 avm(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('avmu', <class 'netCDF4._netCDF4.Variable'>
float64 avmu(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('avmv', <class 'netCDF4._netCDF4.Variable'>
float64 avmv(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
fill

In [3]:
# read z, T, S into CTD
CTD=np.loadtxt('/data/eolson/SOG/SOG-initial/ctd/SG-S3-2004-10-19.sog',skiprows=12,usecols=(1, 4, 2, 8))
# columns are   0:z  1:chl  2:T  3:Si

In [4]:
print(TS.variables[zvar][:])

[   0.5000003     1.5000031     2.50001144    3.50003052    4.50007057
    5.50015068    6.50031042    7.50062323    8.50123596    9.50243282
   10.50476551   11.50931168   12.51816654   13.53541183   14.56898212
   15.63428783   16.76117325   18.00713539   19.48178482   21.38997841
   24.10025597   28.22991562   34.68575668   44.51772308   58.48433304
   76.58558655   98.06295776  121.86651611  147.08946228  173.11448669
  199.57304382  226.26029968  253.06663513  279.93453979  306.834198
  333.75018311  360.67453003  387.60321045  414.53408813  441.46609497]


In [5]:
nuts=np.loadtxt('/data/eolson/SOG/SOG-initial/stratogem_nuts/Nuts-S3-2004-10-19.sog', skiprows=12, usecols=(0,1,2))
# columns are  0:z  1:NO  2:Si  

In [6]:
print(CTD.shape)

(237, 4)


In [7]:
# add z=0 at first row, repeating T,S values from next level
data=np.vstack((CTD[0,:],CTD))
data[0,0]=0.
zs=TS.variables[zvar][:]

In [8]:
# Remove records with negative data values (typically -99.0 or
# -99999) because that indicates invalid data
data_qty=nuts[:,1] #NO
mask = (data_qty >= 0.0)
data_records = len(data_qty[mask])
qty_clean = data_qty[mask]
depth_clean= nuts[:,0][mask]
# Calculate depth and quantity differences from field data for use
NO=np.interp(zs,depth_clean,qty_clean)
print(NO)

[  9.20829982   8.60289812   7.99749307   7.39208152   6.78665728
   6.54051703   6.65353508   6.76657042   6.87963966   6.99277491
   7.54384687   8.5286537    9.51768457  10.51494099  11.52820163
  12.57257408  13.67731619  14.89879518  16.34446775  18.21516533
  20.87218594  24.92069778  26.656       26.656       26.656       26.656
  26.656       26.656       26.656       26.656       26.656       26.656
  26.656       26.656       26.656       26.656       26.656       26.656
  26.656       26.656     ]


In [9]:
# Remove records with negative data values (typically -99.0 or
# -99999) because that indicates invalid data
data_qty=nuts[:,2] #Si
mask = (data_qty >= 0.0)
data_records = len(data_qty[mask])
qty_clean = data_qty[mask]
depth_clean= nuts[:,0][mask]
Si=np.interp(zs,depth_clean,qty_clean)
print (Si)

[ 26.49739951  24.86819495  23.23898136  21.60975028  19.98048502
  19.17070142  19.18010292  19.18950586  19.19891162  19.20832287
  19.95482863  21.4311599   22.91382346  24.408818    25.92780458
  27.49343112  29.14955826  30.98068653  33.14790506  35.95228177
  39.93544118  46.00459549  48.606       48.606       48.606       48.606
  48.606       48.606       48.606       48.606       48.606       48.606
  48.606       48.606       48.606       48.606       48.606       48.606
  48.606       48.606     ]


In [10]:
# Remove records with negative data values (typically -99.0 or
# -99999) because that indicates invalid data
data_qty=data[:,1] # chl
mask = (data_qty >= 0.0)
data_records = len(data_qty[mask])
qty_clean = data_qty[mask]
depth_clean= data[:,0][mask]
P=np.interp(zs,depth_clean,qty_clean)
P[zs>150]=0.0
print (P)

[ 3.8466      3.77219954  3.46309463  3.31560532  3.73684714  4.27036012
  4.45914339  4.5218413   4.09040576  3.26508496  2.74198331  2.56332526
  2.54226848  2.38431421  2.13442678  2.03313374  1.96380502  1.81095658
  1.67864778  1.20629758  0.88869812  0.75623901  0.6288632   0.27409216
  0.17247887  0.14814144  0.12857086  0.09962737  0.086       0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.        ]


In [11]:
# create temp and sal arrays with correct dimensions
vecNO=np.reshape(NO,(zlen,1,1))
data_NO=np.tile(vecNO,(1,1,ylen,xlen))
vecSi=np.reshape(Si,(zlen,1,1))
data_Si=np.tile(vecSi,(1,1,ylen,xlen))
print (data_Si.shape)
vecP=0.33*np.reshape(P,(zlen,1,1))/N2chl
data_PHY=np.tile(vecP,(1,1,ylen,xlen))
data_PHY2=data_PHY
data_MYRI=data_PHY*1e-6
data_MICZ=data_MYRI
data_NH4=0.0*data_PHY+1.0
data_POC=data_PHY2/5.0*1e-6
data_DOC=data_POC/10.0
data_bSi=data_POC
data_POC=data_POC*7.6
data_DOC=data_DOC*7.6
print (data_NH4[0,0,:,:])

(1, 40, 898, 398)
[[ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 ..., 
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]]


In [12]:
new_nuts=nc.Dataset('/ocean/eolson/MEOPAR/NEMO-3.6-inputs/initial_conditions/nuts_SOG_fullDomain.nc','w')
new_nuts.createDimension('y', ylen)
new_nuts.createDimension('x', xlen)
new_nuts.createDimension('z', zlen)
new_nuts.createDimension('t', None)

print(TS)
print(new_nuts)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format UNDEFINED):
    file_name: SalishSea_02376000_restart.nc
    TimeStamp: 11/06/2015 12:39:04 -0700
    history: Thu Jun 11 12:45:31 2015: ncks -4 -L4 -O SalishSea_02376000_restart.nc SalishSea_02376000_restart.nc
    NCO: 4.4.2
    dimensions(sizes): t(1), z(40), y(898), x(398)
    variables(dimensions): float64 adatrj(), float64 avm(t,z,y,x), float64 avmu(t,z,y,x), float64 avmv(t,z,y,x), float64 avt(t,z,y,x), float64 emp_b(t,y,x), float64 emps_b(t,y,x), float64 en(t,z,y,x), float64 fse3t_b(t,z,y,x), float64 hdivb(t,z,y,x), float64 hdivn(t,z,y,x), float64 kt(), float64 mxln(t,z,y,x), float32 nav_lat(y,x), float32 nav_lev(z), float32 nav_lon(y,x), float64 ndastp(), float64 qns_b(t,y,x), float64 qsr_hc_b(t,z,y,x), float64 rdt(), float64 rdttra1(), float64 rhop(t,z,y,x), float64 rnf_b(t,y,x), float64 rnf_hc_b(t,y,x), float64 rnf_sc_b(t,y,x), float64 rotb(t,z,y,x), float64 rotn(t,z,y,x), float64 sb(t,z,y,x), floa

In [13]:
new_tc=new_nuts.createVariable(tvar,float,('t'),zlib=True)
#new_tc.setncattr('units',TS.variables[tvar].units)
#new_tc.setncattr('long_name',TS.variables[tvar].long_name)
new_tc[:]=TS.variables[tvar]

print (TS.variables[tvar])
print( new_nuts.variables[tvar])

<class 'netCDF4._netCDF4.Variable'>
float64 time_counter(t)
unlimited dimensions: t
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 time_counter(t)
unlimited dimensions: t
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used



In [14]:
new_z=new_nuts.createVariable(zvar,float,('z'),zlib=True)
#new_z.setncattr('units',TS.variables[zvar].units)
#new_z.setncattr('long_name',TS.variables[zvar].long_name)
#new_z.setncattr('positive',TS.variables[zvar].positive)
new_z[:]=zs

print( TS.variables[zvar])
print (new_nuts.variables[zvar])

<class 'netCDF4._netCDF4.Variable'>
float32 nav_lev(z)
unlimited dimensions: 
current shape = (40,)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 nav_lev(z)
unlimited dimensions: 
current shape = (40,)
filling on, default _FillValue of 9.969209968386869e+36 used



In [15]:
new_x=new_nuts.createVariable(xvar,float,('y','x'),zlib=True)
#new_x.setncattr('units',TS.variables[xvar].units)
#new_x.setncattr('long_name',TS.variables[xvar].long_name)
new_x=TS.variables[xvar]

print (TS.variables[xvar])
print (new_nuts.variables[xvar])

<class 'netCDF4._netCDF4.Variable'>
float32 nav_lon(y, x)
unlimited dimensions: 
current shape = (898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 nav_lon(y, x)
unlimited dimensions: 
current shape = (898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [16]:
new_y=new_nuts.createVariable(yvar,float,('y','x'),zlib=True)
#new_y.setncattr('units',TS.variables[yvar].units)
#new_y.setncattr('long_name',TS.variables[yvar].long_name)
new_y=TS.variables[yvar]

print( TS.variables[yvar])
print( new_nuts.variables[yvar])

<class 'netCDF4._netCDF4.Variable'>
float32 nav_lat(y, x)
unlimited dimensions: 
current shape = (898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 nav_lat(y, x)
unlimited dimensions: 
current shape = (898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [17]:
new_NO=new_nuts.createVariable('NO3',float,('t','z','y','x'),zlib=True)
#new_Tem.setncattr('units',TS.variables['NO3'].units)
#new_NO.setncattr('long_name','Nitrate')
#new_NO.setncattr('coordinates',TS.variables['votemper'].coordinates)
new_NO[:,:,:,:]=data_NO
print( TS.variables['sn'])
print( new_nuts.variables['NO3'])

<class 'netCDF4._netCDF4.Variable'>
float64 sn(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 NO3(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [18]:
new_Si=new_nuts.createVariable('Si',float,('t','z','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
#new_Si.setncattr('long_name','Silicate')
#new_Si.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_Si[:,:,:,:]=data_Si
print( new_nuts.variables['Si'])

<class 'netCDF4._netCDF4.Variable'>
float64 Si(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [19]:
new_NH4=new_nuts.createVariable('NH4',float,('t','z','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
#new_NH4.setncattr('long_name','Ammonium')
#new_NH4.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_NH4[:,:,:,:]=data_NH4
print (new_nuts.variables['NH4'])

<class 'netCDF4._netCDF4.Variable'>
float64 NH4(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [20]:
new_PHY=new_nuts.createVariable('PHY',float,('t','z','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
#new_PHY.setncattr('long_name','PHY')
#new_PHY.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_PHY[:,:,:,:]=data_PHY
print (new_nuts.variables['PHY'])

<class 'netCDF4._netCDF4.Variable'>
float64 PHY(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [21]:
new_PHY2=new_nuts.createVariable('PHY2',float,('t','z','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
#new_PHY2.setncattr('long_name','PHY2')
#new_PHY2.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_PHY2[:,:,:,:]=data_PHY2
print( new_nuts.variables['PHY2'])

<class 'netCDF4._netCDF4.Variable'>
float64 PHY2(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [22]:
new_MYRI=new_nuts.createVariable('MYRI',float,('t','z','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
#new_MYRI.setncattr('long_name','MYRI')
#new_MYRI.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_MYRI[:,:,:,:]=data_MYRI
print( new_nuts.variables['MYRI'])

<class 'netCDF4._netCDF4.Variable'>
float64 MYRI(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [23]:
new_MICZ=new_nuts.createVariable('MICZ',float,('t','z','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
#new_MICZ.setncattr('long_name','MICZ')
#new_MICZ.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_MICZ[:,:,:,:]=data_MICZ
print( TS.variables['sn'])
print (new_nuts.variables['MICZ'])

<class 'netCDF4._netCDF4.Variable'>
float64 sn(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 MICZ(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [24]:
new_POC=new_nuts.createVariable('POC',float,('t','z','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
#new_POC.setncattr('long_name','POC')
#new_POC.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_POC[:,:,:,:]=data_POC
print (new_nuts.variables['POC'])

<class 'netCDF4._netCDF4.Variable'>
float64 POC(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [25]:
new_DOC=new_nuts.createVariable('DOC',float,('t','z','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
#new_DOC.setncattr('long_name','DOC')
#new_DOC.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_DOC[:,:,:,:]=data_DOC
print( new_nuts.variables['DOC'])

<class 'netCDF4._netCDF4.Variable'>
float64 DOC(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [26]:
new_bSi=new_nuts.createVariable('bSi',float,('t','z','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
#new_bSi.setncattr('long_name','bSi')
#new_bSi.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_bSi[:,:,:,:]=data_bSi
print (new_nuts.variables['bSi'])

<class 'netCDF4._netCDF4.Variable'>
float64 bSi(t, z, y, x)
unlimited dimensions: t
current shape = (1, 40, 898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used



In [27]:
new_nuts.title="""SalishSea SOG full domain NO, Si, NH4, PHY, PHY2, MYRI, Z, DOC, POC, bSi initialization"""
new_nuts.institution="""
Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia"""
new_nuts.comment= """
Based on SG-S3-2004-10-19.sog and nuts-S3-2004-10-19.sog"""
new_nuts.reference= """
eolson: createIC_NutsPhy_fullDomain.ipynb"""

In [28]:
new_nuts.close()